# 5.11 (b)
## TensorFlow スタイルによるプログラミング

<a href="../html/tf2_book4_ch05_01b.html">「5.1 tf.keras.layers.Conv2DによるCNNの実装」TensorFlowスタイル</a>
をRNNを使うように書き換える。


In [1]:
# 5-11-6 Fashion-MNIST データの読み込みと前処理
# p.389
# [自分へのメモ] 混乱を避けるため、変数の値を書き換えるのではなく、別の変数に保存することにする

import tensorflow as tf
from tensorflow  import keras

(x_train, t_train), (x_test, t_test) = tf.keras.datasets.fashion_mnist.load_data()

x_train_norm = x_train.astype('float32') / 255.
x_test_norm = x_test.astype('float32') / 255.

In [4]:
# 5-11-7 RNN モデルの構築
# p.390
# [自分へのメモ] 入力データの形状も指定できるように変更した

class RNN(keras.Model):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.i = keras.layers.InputLayer(input_shape=input_dim)
        weight_decay = 1e-4
        self.l1 = keras.layers.LSTM(units=128, dropout=0.25, return_sequences=True)
        self.l2 = keras.layers.LSTM(units=128, dropout=0.25, return_sequences=True)
        self.l3 = keras.layers.LSTM(units=128, dropout=0.5, 
                                    return_sequences=False,   ### !!!
                                    kernel_regularizer=keras.regularizers.l2(weight_decay))
        self.o = keras.layers.Dense(units=10, activation='softmax')
        self.ls = [ self.l1, self.l2, self.l3, self.o ]
        
    def call(self, x):
        for layer in self.ls:
            x = layer(x)
        return x

In [5]:
model = RNN(x_train_norm[0].shape, 10)

In [6]:
# 5-01-3 損失関数の定義
# p.263

cce = keras.losses.SparseCategoricalCrossentropy()

def loss(t, y):
    return cce(t,y)

In [7]:
# 5-01-4 Stochastic Gradient Descend を使用するオプティマイザー
# p.263
# [自分へのメモ] Adam()に変更した。

optimizer = keras.optimizers.Adam()

# 損失を記録するオブジェクト
train_loss = keras.metrics.Mean()

# 精度を記録するオブジェクト
train_acc = keras.metrics.SparseCategoricalAccuracy()

# validationの損失を記録するオブジェクト
val_loss = keras.metrics.Mean()

# validationの精度を記録する
val_acc = keras.metrics.SparseCategoricalAccuracy()

In [8]:
import tensorflow as tf
# p.264
def train_step(x, t):
    '''
    学習を1回行う
    
    Parameters:
      x (ndarray(float32)): 訓練データ
      t (ndarray(float32)): 正解ラベル
      
    Returns:
      ステップごとのクロスエントロピー誤差
      
      '''
    
    with tf.GradientTape() as tape:
        outputs = model(x)
        tmp_loss = loss(t, outputs)
    grads = tape.gradient(tmp_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    # 損失と精度を記録
    train_loss(tmp_loss)
    train_acc(t, outputs)

In [9]:
# 5-01-5 検証データによる評価を行う
# p.265
def val_step(x, t):
    preds = model(x)
    tmp_loss = loss(t, preds)
    val_loss(tmp_loss)
    val_acc(t, preds)

In [10]:
# 5-01-6 訓練データを検証データを用意する
# p.266
from sklearn.model_selection import train_test_split

x_tr, x_val, t_tr, t_val = train_test_split(x_train_norm, t_train, test_size=0.2)

In [11]:
# 5-01-7 モデルを生成して学習する
# p.266
from sklearn.utils import shuffle

epochs = 100
batch_size = 64
steps = x_train.shape[0] // batch_size
steps_val = x_val.shape[0] // batch_size

In [13]:
# 学習を行う
# p.267

for epoch in range(epochs):
    x_, t_ = shuffle(x_tr, t_tr, random_state=1)
    
    for step in range(steps):
        start = step * batch_size
        end = start + batch_size
        train_step(x_[start:end], t_[start:end])
        
    for step_val in range(steps_val):
        start = step_val * batch_size
        end = start + batch_size
        val_step(x_val[start:end], t_val[start:end])
        
    print(f'epoch: {(epoch+1)} '
          f'train_loss: {train_loss.result():.4f} '
          f'train_acc:{train_acc.result():.4f} '
          f'val_loss: {val_loss.result():.4f} '
          f'val_acc: {val_acc.result():.4f}')

InvalidArgumentError: Value for attr 'T' of uint8 is not in the list of allowed values: half, float, double
	; NodeDef: {{node CudnnRNN}}; Op<name=CudnnRNN; signature=input:T, input_h:T, input_c:T, params:T -> output:T, output_h:T, output_c:T, reserve_space:T; attr=T:type,allowed=[DT_HALF, DT_FLOAT, DT_DOUBLE]; attr=rnn_mode:string,default="lstm",allowed=["rnn_relu", "rnn_tanh", "lstm", "gru"]; attr=input_mode:string,default="linear_input",allowed=["linear_input", "skip_input", "auto_select"]; attr=direction:string,default="unidirectional",allowed=["unidirectional", "bidirectional"]; attr=dropout:float,default=0; attr=seed:int,default=0; attr=seed2:int,default=0; attr=is_training:bool,default=true; is_stateful=true> [Op:CudnnRNN]